In [1]:
import cudf
import pyarrow as pa
import pandas as pd
import numpy as np
import cupy as cp

In [2]:
temp = cudf.read_parquet('./temp.parquet/*')[['p_id','GISJOIN','JLZE002','JLZE026']]

/home/ajay/anaconda3/envs/rapids_dev_latest/lib/python3.7/site-packages/fsspec/implementations/local.py:33: FutureWarning: The default value of auto_mkdir=True has been deprecated and will be changed to auto_mkdir=False by default in a future release.
  FutureWarning,


In [3]:
temp

,p_id,GISJOIN,JLZE002,JLZE026
0,5280,1000300112024031,0.500792,0.499208
1,5281,1000300112024031,0.500792,0.499208
2,5282,1000300112024031,0.500792,0.499208
3,5283,1000300112024031,0.500792,0.499208
4,5284,1000300112024031,0.500792,0.499208
...,...,...,...,...
308745304,308735867,56004300003013056,0.445183,0.554817
308745305,308735868,56004300003013056,0.445183,0.554817
308745306,308735869,56004300003013056,0.445183,0.554817
308745307,308735870,56004300003013056,0.445183,0.554817


In [4]:
temp1 = temp[['GISJOIN','JLZE002','JLZE026']].groupby(by='GISJOIN').mean()

In [5]:
temp2 = temp[['GISJOIN','JLZE002','JLZE026']].groupby(by='GISJOIN').count()

temp1.JLZE002 = temp1.JLZE002
temp1.JLZE026 = temp1.JLZE026
temp1 = temp1.nans_to_nulls()
temp1 = temp1.fillna(0.5)

In [6]:
del(temp)

In [7]:
temp1['total_count'] = temp2.JLZE002

In [8]:
del(temp2)

In [9]:
temp1['Male'] = (temp1.JLZE002 * temp1.total_count).round().astype('int')

In [10]:
temp1['Female'] = (temp1.total_count - temp1.Male).astype('int')

In [11]:
temp1

,JLZE002,JLZE026,total_count,Male,Female
GISJOIN,,,,,
1000100201001000,0.427788,0.572212,61,26,35
1000100201001003,0.427788,0.572212,75,32,43
1000100201001005,0.427788,0.572212,1,0,1
1000100201001007,0.427788,0.572212,23,10,13
1000100201001009,0.427788,0.572212,1,0,1
...,...,...,...,...,...
56004509513003096,0.536510,0.463490,12,6,6
56004509513003104,0.536510,0.463490,160,86,74
56004509513003112,0.536510,0.463490,3,2,1


In [12]:
(temp1.Male.sum() + temp1.Female.sum()) == temp1.total_count.sum()

True

In [13]:
temp1 = temp1.drop(['JLZE002', 'JLZE026', 'total_count'])

In [14]:
temp1

,Male,Female
GISJOIN,,
1000100201001000,26,35
1000100201001003,32,43
1000100201001005,0,1
1000100201001007,10,13
1000100201001009,0,1
...,...,...
56004509513003096,6,6
56004509513003104,86,74
56004509513003112,2,1


In [22]:
df_males = cudf.DataFrame({
    'GISJOIN': temp1.index.repeat(temp1.Male).values
})
df_males['sex'] = 0
df_females = cudf.DataFrame({
    'GISJOIN': temp1.index.repeat(temp1.Female).values
})
df_females['sex'] = 1

In [23]:
df_final = cudf.concat([df_males, df_females]).reset_index()[['GISJOIN','sex']]
del(df_males)
del(temp1)
del(df_females)

In [24]:
# df_final.index= df_final.GISJOIN
# df_final.drop_column('GISJOIN')
df_final

,GISJOIN,sex
0,1000100201001000,0
1,1000100201001000,0
2,1000100201001000,0
3,1000100201001000,0
4,1000100201001000,0
...,...,...
308745304,56004509513003128,1
308745305,56004509513003128,1
308745306,56004509513003128,1
308745307,56004509513003128,1


In [25]:
df_final.to_parquet('temp2.parquet')

### sort temp.parquet

In [2]:
df = cudf.read_parquet('./temp.parquet/*')[['GISJOIN', 'x', 'y']]
df = df.sort_values('GISJOIN')
df.to_parquet('sorted_temp.parquet')

/home/ajay/anaconda3/envs/rapids_dev_latest/lib/python3.7/site-packages/fsspec/implementations/local.py:33: FutureWarning: The default value of auto_mkdir=True has been deprecated and will be changed to auto_mkdir=False by default in a future release.
  FutureWarning,


In [4]:
df = cudf.read_parquet('./temp2.parquet')[['GISJOIN', 'sex']]
df = df.sort_values('GISJOIN')
df.to_parquet('sorted_temp2.parquet')

### merge sex with lat-long table

In [1]:
import cudf
import pyarrow as pa
import pandas as pd
import numpy as np
import cupy as cp

In [2]:
df = cudf.read_parquet('./sorted_temp.parquet')

/home/ajay/anaconda3/envs/rapids_dev_latest/lib/python3.7/site-packages/fsspec/implementations/local.py:33: FutureWarning: The default value of auto_mkdir=True has been deprecated and will be changed to auto_mkdir=False by default in a future release.
  FutureWarning,


In [3]:
df.head(100)

,GISJOIN,x,y
0,1000100201001000,-86.480965,32.468025
1,1000100201001000,-86.484355,32.469958
2,1000100201001000,-86.476974,32.467935
3,1000100201001000,-86.483532,32.468555
4,1000100201001000,-86.479665,32.471680
...,...,...,...
95,1000100201001003,-86.492626,32.471707
96,1000100201001003,-86.480582,32.464173
97,1000100201001003,-86.488611,32.469550
98,1000100201001003,-86.489743,32.470649


In [4]:
df_with_sex = cudf.read_parquet('./sorted_temp2.parquet').reset_index().drop(['index'])

In [5]:
df_with_sex.head(100)

,GISJOIN,sex
0,1000100201001000,0
1,1000100201001000,0
2,1000100201001000,0
3,1000100201001000,0
4,1000100201001000,0
...,...,...
95,1000100201001003,1
96,1000100201001003,1
97,1000100201001003,1
98,1000100201001003,1


In [6]:
df['GISJOIN1'] = df_with_sex.GISJOIN.values

In [7]:
df.query('GISJOIN != GISJOIN1')

,GISJOIN,x,y,GISJOIN1


In [8]:
df['sex'] = df_with_sex.sex.astype('int8')

In [9]:
df.drop_column('GISJOIN1')

In [10]:
df.dtypes

GISJOIN      int64
x          float64
y          float64
sex           int8
dtype: object

In [11]:
df

,GISJOIN,x,y,sex
0,1000100201001000,-86.480965,32.468025,0
1,1000100201001000,-86.484355,32.469958,0
2,1000100201001000,-86.476974,32.467935,0
3,1000100201001000,-86.483532,32.468555,0
4,1000100201001000,-86.479665,32.471680,0
...,...,...,...,...
308745304,56004509513003128,-104.197455,43.845559,1
308745305,56004509513003128,-104.198425,43.845850,1
308745306,56004509513003128,-104.199305,43.846296,1
308745307,56004509513003128,-104.199264,43.846305,1


In [12]:
df = df.sort_values('GISJOIN')

In [13]:
del(df_with_sex)

In [14]:
df

,GISJOIN,x,y,sex,p_id
0,1000100201001000,-86.480965,32.468025,0,0
1,1000100201001000,-86.484355,32.469958,0,1
2,1000100201001000,-86.476974,32.467935,0,2
3,1000100201001000,-86.483532,32.468555,0,3
4,1000100201001000,-86.479665,32.471680,0,4
...,...,...,...,...,...
308745304,56004509513003128,-104.197455,43.845559,1,308745304
308745305,56004509513003128,-104.198425,43.845850,1,308745305
308745306,56004509513003128,-104.199305,43.846296,1,308745306
308745307,56004509513003128,-104.199264,43.846305,1,308745307


In [15]:
df.to_parquet('../../data/all_states_us_lat_long_sex.parquet')

### reset index

In [1]:
import cudf

df = cudf.read_parquet('../../data/all_states_us_lat_long_sex.parquet')

df = df.reset_index()
df.drop_column('index')
df['p_id'] = df.index.astype('int32')

/home/ajay/anaconda3/envs/rapids_dev_latest/lib/python3.7/site-packages/fsspec/implementations/local.py:33: FutureWarning: The default value of auto_mkdir=True has been deprecated and will be changed to auto_mkdir=False by default in a future release.
  FutureWarning,


In [2]:
df.to_parquet('../../data/all_states_us_lat_long_sex.parquet')